In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from math import sqrt
import time

from skimage.io import imread, imshow, imread_collection
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure

from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, RFE, RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import KFold
from google.colab import drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

train_img = '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/train_resized_100'
train_csv = '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/train.csv'
test_img = '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/test_resized_100'
test_csv = '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/test.csv'

#wrapper model
pkl_wrapper_model = '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/RFEcv_model.pkl'





In [ ]:
# train data with hog feature extraction applied
hogged_data = pd.read_csv("/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/hogged_data.csv")

In [ ]:
train_X = hogged_data.iloc[:,0:-1].values
train_Y = hogged_data.iloc[:,-1].values




In [ ]:
#tempoary code
xray_image_location = '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/covid_images_from_torchxrayvision'


path, dirs, files = next(os.walk(xray_image_location))

labels =[]
images_names = []
for image_name in files:
  labels.append('1')
  images_names.append(image_name)
d = {'filename':images_names,'covid(label)':labels}

covid_torchxrayvision = pd.DataFrame(d)






In [ ]:
# covid_torchxrayvision.to_csv('/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/covid_torchxrayvision.csv',index=False)

# Filter methods

In [ ]:

bestfeatures_filter = SelectKBest(score_func=chi2, k=200)
filter_fit = bestfeatures_filter.fit(train_X,train_Y)
filter_indices=filter_fit.get_support(indices = True)
train_data = pd.DataFrame()
for feature in filter_indices:
  train_data[str(feature)] = hogged_data[str(feature)] 
train_X = train_data.values
train_Y = hogged_data.iloc[:,-1].values
# adaboost code
kf = KFold(n_splits=5)
kf.get_n_splits(train_X)
adaboost = AdaBoostClassifier(n_estimators=100, base_estimator= None,learning_rate=1, random_state = 1)
accuracy =[]
for train_index, test_index in kf.split(train_X):
  train_x, test_x = train_X[train_index], train_X[test_index]
  train_y, test_y = train_Y[train_index], train_Y[test_index]
  adaboost.fit(train_x, train_y)
  accuracy.append(adaboost.score(test_x,test_y))
print('Accuracy of adaboost when using filter feature = {} %'.format(sum(accuracy)*100/len(accuracy)))
train_X = hogged_data.iloc[:,0:-1].values
train_Y = hogged_data.iloc[:,-1].values



#  Boosting is an Ensemble learning methods that makes use of multiple learners. In boosting, different algorithms work closer by learning from each other. 

# Adaboost on hogged data

In [ ]:
# adaboost code
kf = KFold(n_splits=5)
kf.get_n_splits(train_X)
adaboost = AdaBoostClassifier(n_estimators=100, base_estimator= None,learning_rate=1, random_state = 1)



In [ ]:
accuracy =[]
for train_index, test_index in kf.split(train_X):
  train_x, test_x = train_X[train_index], train_X[test_index]
  train_y, test_y = train_Y[train_index], train_Y[test_index]
  adaboost.fit(train_x, train_y)
  accuracy.append(adaboost.score(test_x,test_y))



In [ ]:
#Accuracy of adaboost when using feature from wrapper
print('Accuracy of adaboost on features after hog feature extraction= {} %'.format(sum(accuracy)*100/len(accuracy)))


# Wrapper feature with Adaboost


In [ ]:
#get data from wrapper model
import pickle
with open(pkl_wrapper_model, 'rb') as f:
     wrapper_model = pickle.load(f)

In [ ]:
print("Optimal number of features : %d" % wrapper_model.n_features_)
print("RFE_CV indices:", wrapper_model.get_support(indices = True))

In [ ]:
print(wrapper_model)

In [ ]:
feature_indices = wrapper_model.get_support(indices = True)
print(feature_indices)

In [ ]:
train_data = pd.DataFrame()
for feature in feature_indices:
  train_data[str(feature)] = hogged_data[str(feature)] 
train_X = train_data.values
train_Y = hogged_data.iloc[:,-1].values


In [ ]:
# adaboost code
kf = KFold(n_splits=5)

# hogged data and wrapper columsn
# kf.get_n_splits(train_X)

kf.get_n_splits(X_train)


adaboost = AdaBoostClassifier(n_estimators=100, base_estimator= None,learning_rate=1, random_state = 1)



In [ ]:
display(feature_indices)
display(len(X_train))

In [ ]:
# wrapper features adaboost
accuracy =[]
for train_index, test_index in kf.split(train_X):
  train_x, test_x = train_X[train_index], train_X[test_index]
  train_y, test_y = train_Y[train_index], train_Y[test_index]
  adaboost.fit(train_x, train_y)
  accuracy.append(adaboost.score(test_x,test_y))




In [ ]:
display(X_train.columns.values)

In [ ]:
# not. wrapper features adaboost
train_X = X_train.values
accuracy =[]
for train_index, test_index in kf.split(train_X):
  train_x, test_x = train_X[train_index], train_X[test_index]
  train_y, test_y = train_Y[train_index], train_Y[test_index]
  adaboost.fit(train_x, train_y)
  accuracy.append(adaboost.score(test_x,test_y))




In [ ]:
#Accuracy of adaboost when using feature from wrapper
print('Accuracy of adaboost when using feature from wrapper = {} %'.format(sum(accuracy)*100/len(accuracy)))


In [ ]:
# display(X_train)
# display(X_train.columns.values)


In [ ]:
! pip install torchxrayvision

In [ ]:
import torchxrayvision as xrv

In [ ]:
! git clone https://github.com/ieee8023/covid-chestxray-dataset


In [ ]:
d_covid19 = xrv.datasets.COVID19_Dataset(imgpath='/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/covid-chestxray-dataset/images', csvpath='/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/covid-chestxray-dataset/metadata.csv')
csvpath_for_torchxrayvision = '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/covid-chestxray-dataset/metadata.csv'
imgpath_for_torchxrayvision = '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/covid-chestxray-dataset/images'



In [ ]:
display(d_covid19)

In [ ]:
#reading the image
img_1 = imread('/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/covid-chestxray-dataset/images/16858_1_1.png',as_gray=True)
imshow(img_1)
print(img_1.shape)

In [ ]:
xray_csv = pd.read_csv(csvpath_for_torchxrayvision)
df_covidcsv = xray_csv.loc[xray_csv['finding'] == 'Pneumonia/Viral/COVID-19']

In [ ]:
#make a list of names which are covid images 
# display(df_covidcsv)
# df_covidcsv['covid(label)'] = np.where(df_covidcsv['RT_PCR_positive']!= 'Y', 0, 1)
# df_covidcsv = df_covidcsv.drop(columns=['covid(label)'])
# len(df_covidcsv)
# print(len(df_covidcsv.loc[df_covidcsv['RT_PCR_positive']=='Pneumonia/Viral/COVID-19']))
display(df_covidcsv.columns.values)
df_covidcsv['age']= df_covidcsv['age'].fillna(df_covidcsv['age'].mean())
# df_covidcsv[['age','filename','sex', 'covid(label)']].to_csv('/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/train_torchxrayvision.csv',index=False)


In [ ]:
!mkdir '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/covid_images_from_torchxrayvision/'

In [ ]:
! ls '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/'

In [ ]:
import os
for name in covid_images_list:
  cmd = 'cp \'/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/covid-chestxray-dataset/images/\'' +str(name)+ ' \'/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/covid_images_from_torchxrayvision/.\''
  print(cmd)
  os.system(cmd)  



In [ ]:
! cd '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/'

In [ ]:
! ls '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/covid_images_from_torchxrayvision/'

# Improving performance

In [ ]:
train_csv_path = '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/train.csv'
train_X = hogged_data.iloc[:,0:-1]
train_Y = hogged_data.iloc[:,-1]
train_csv = pd.read_csv(train_csv_path)

In [ ]:
def get_wrapper_features_dataset(train_X,feature_indices):
  train_data = pd.DataFrame()
  for feature in feature_indices:
    train_data[str(feature)] = hogged_data[str(feature)] 
  train_X = train_data
  return train_X

train_X = get_wrapper_features_dataset(train_X,feature_indices)


In [ ]:
# joinind data age, gender and location

df_oneencode_location = pd.get_dummies(train_csv['location'],prefix='location')
df_oneencode_gender = pd.get_dummies(train_csv['gender'],prefix='gender')

In [ ]:
train_csv = pd.concat([train_csv, df_oneencode_location], axis=1, sort=False)

In [ ]:
train_csv = pd.concat([train_csv, df_oneencode_gender], axis=1, sort=False)

In [ ]:
train_csv = train_csv.drop(['filename','covid(label)','location','gender'],axis =1)


In [ ]:
X_train = pd.concat([train_csv, train_X], axis=1, sort=False)

In [ ]:
display(X_train)
X_train['age'] = X_train['age'].fillna(X_train['age'].mean(skipna = True) )

In [ ]:
len(X_train.values[1])

In [ ]:
import tensorflow as tf
from math import sqrt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.regularizers import l2
from keras.layers import Dense, Dropout

#  Define a Feed-Forward Model with 2 hidden layers with dimensions 1152 and 576 Neurons
model = Sequential([
  Dense(179, activation='relu', input_shape=(179,), name="first_hidden_layer"),
  Dense(179//2, activation='relu', name="second_hidden_layer"),Dropout(0.25),
  Dense(2, activation='softmax'),
])
print(model.summary())


# compile model
model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'],)


In [ ]:
from time import time
start = time()
performance = model.fit(X_train, to_categorical(train_Y), epochs=20, batch_size=256,)
print('Training time for this model is = ', time()-start)

In [ ]:
type(train_Y)

# K-means clustering

In [ ]:
for feature in list(X_train.columns.values):
  X_train[feature].fillna((X_train[feature].mean()), inplace=True)


In [ ]:
from sklearn.cluster import KMeans


In [ ]:
X_train = X_train.values
def cross_val(X_val,train_Y):
  y=train_Y
  score = []
  skf = KFold(n_splits=5) 
  for train_index, test_index in skf.split(X_val):
    x_train, x_test = X_val[train_index], X_val[test_index]
    y_train, y_test = y[train_index], y[test_index]
    kmeans = KMeans(n_clusters=2, random_state=0)
    kmeans.fit(x_train, y_train)
    y_labels_test = kmeans.predict(x_test)
    count =0
    for i,j in zip(y_labels_test,y_test):
      if i==j:
        count =count+1
    print("accuracy:", count*100/len(y_labels_test) )



    # print(y_labels_test)

    # score.append(kmeans.score(x_test, y_test))

  return 


print(cross_val(X_train,train_Y))
